# sequence prediction

In [1]:
import tensorflow as tf
from tensorflow import nn
import numpy as np
import math

lstm_size  = 512
n_layers   = 3
n_epochs   = 1000
seq_len    = 50
batch_size = 50
lr = 1
grad_norm = 5
save_freq = 10

## utils

In [2]:
def xavier(n_in, n_out):
    init_range = 4*math.sqrt(6.0/(n_in + n_out))
    return tf.random_uniform([n_in, n_out], -init_range, init_range)

## data loading

In [3]:
with open('shakespeare.txt') as f:
    words_full = [c for c in list(f.read())]
    vocab = []
    for c in words_full:
        if c not in vocab:
            vocab.append(c)
    vocab_size = len(vocab)    
    
    #make batches
    tr_x = np.array([vocab.index(el) for el in words_full], dtype='int32')
    #target is just data shifted one element to the left
    tr_y = np.roll(tr_x, -1)
    sp = [i*batch_size*seq_len for i in range(1,len(words_full)//(batch_size*seq_len) + 1)]
    n_batches = len(tr_x)//(batch_size*seq_len)
    if(len(tr_x) % batch_size*seq_len != 0):
        # drop last small batch
        tr_x = np.array(np.split(tr_x, sp)[:-1]).reshape(n_batches, seq_len, batch_size)
        tr_y = np.array(np.split(tr_y, sp)[:-1]).reshape(n_batches, seq_len, batch_size)

## model

In [4]:
#create model
#tf.reset_default_graph()

lstm = nn.rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = nn.rnn_cell.MultiRNNCell([lstm] * n_layers)
input_data = tf.placeholder(tf.int32, [batch_size, seq_len])
targets = tf.placeholder(tf.int32, [batch_size, seq_len])
initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)
embedding = tf.get_variable("embedding", [vocab_size, lstm_size])
inputs = tf.nn.embedding_lookup(embedding, input_data)

outputs = []
for i in range(seq_len):
    if i > 0:
        tf.get_variable_scope().reuse_variables()
    cell_output, state = stacked_lstm(inputs[:,i,:], state)
    outputs.append(cell_output)
final_state = state

output = tf.reshape(tf.concat(1, outputs), [-1, lstm_size])
softmax_w = tf.Variable(xavier(lstm_size, vocab_size), dtype=tf.float32)
softmax_b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
z = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(z)
loss = tf.nn.seq2seq.sequence_loss_by_example([z], [tf.reshape(targets, [-1])], [tf.ones([batch_size * seq_len])], vocab_size)
cost = tf.reduce_sum(loss) / batch_size / seq_len
lr = tf.Variable(lr, trainable=False)
optimizer = tf.train.GradientDescentOptimizer(lr)
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_norm)
train_op = optimizer.apply_gradients(zip(grads, tvars))


## training

In [5]:
init = tf.initialize_all_variables()
saver = tf.train.Saver(tf.all_variables())
with tf.Session() as s:
    tf.initialize_all_variables().run()
    state = initial_state.eval()
    for i in range(n_epochs):
        ep_cost = 0
        for j in range(n_batches):
            print("\r", 'Epoch %d, Batch %d' % (i+1, j+1), end='')
            _, _, co, _ = s.run([output, final_state, cost, train_op], feed_dict={input_data:tr_x[j], targets:tr_y[j], initial_state: state})
            ep_cost+=co
        if i % save_freq == 0:
            print("\r", 'Epoch %d, cost %.3f' % (i+1, ep_cost))
            saver.save(s, 'model')

 Epoch 1, cost 228.344
 Epoch 11, cost 118.438
 Epoch 21, cost 94.516
 Epoch 31, cost 85.128
 Epoch 41, cost 78.109
 Epoch 51, cost 71.946
 Epoch 61, cost 65.599
 Epoch 71, cost 57.648
 Epoch 81, cost 47.297
 Epoch 91, cost 34.219
 Epoch 101, cost 18.339
 Epoch 111, cost 7.998
 Epoch 121, cost 6.148
 Epoch 131, cost 5.384
 Epoch 141, cost 4.971
 Epoch 151, cost 4.711
 Epoch 161, cost 4.529


## testing

In [6]:
tf.reset_default_graph()
lstm = nn.rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = nn.rnn_cell.MultiRNNCell([lstm] * n_layers)
input_data = tf.placeholder(tf.int32, [1, 1])
initial_state = state = stacked_lstm.zero_state(1, tf.float32)

embedding = tf.get_variable("embedding", [vocab_size, lstm_size])
inputs = tf.nn.embedding_lookup(embedding, input_data)
cell_output, state = stacked_lstm(inputs[:,0,:], state)
outputs = [cell_output]
final_state = state

output = tf.reshape(tf.concat(1, outputs), [-1, lstm_size])
softmax_w = tf.Variable(tf.random_normal([lstm_size, vocab_size], stddev=0.35), dtype=tf.float32)
softmax_b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
z = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(z)

In [9]:
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    state = initial_state.eval()
    saver = tf.train.Saver(tf.all_variables())
    ckpt = tf.train.get_checkpoint_state('model')
    saver.restore(sess, 'model')

    inpt = 'LEAR'
    sample_size = 1000
    for c in inpt[:-1]:
        x = np.zeros((1,1), dtype='int32')
        x [0,0] = vocab.index(c)
        state = sess.run(final_state, feed_dict={input_data:x, initial_state:state})
    
    res = inpt
    char = inpt[-1]

    for i in range(sample_size):
        x = np.zeros((1,1), dtype='int32')
        x[0, 0] = vocab.index(char)
        probabilities, state = sess.run([probs, final_state], feed_dict={input_data:x, initial_state:state})
        p = probabilities[0]
        sample = np.argmax(p)
        pred = vocab[sample]
        res += pred
        char = pred
    print(res)

LEARENE:
E: sir.

Second Sartert:
The morrach, and us much domner wond and so tear you sere
As it he it to came that them.

BLOTRSSA:
I thou not werke in, lort, preatent, for conselt thou wint
cill thou hear.

SAUK lot Antomita's his distrect comasn in dink come that a fitler.

BUKE HoRY I I make that him: had it it not
I me her that with came again,
To coneer thou shall hoad them our ready, and teat is hipos,
But Too his heard shemind dothere he was ding ears there torre
Co omeerrible stas gan a called out the wort they souldss lederone deador
The know his is 'as inder, to mast my affaces's see,
That in suve seven to tather my throught
Hen leave I longen stach what the hands to fat in lifer,
When shall man his preat in the partifes of a my riftaf; Bthald lord on Purg:
I say all the poon the forth op the our palre,
And fas we shoull for and him to peen usery to your come andre
And sirch and blood ham strein; with be mendition, bry thinoger
Shall Lood the speaken of fece do a rud mistle